In [24]:
import os
import openai
import anthropic
from tqdm import tqdm
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore", module="tqdm")

In [25]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = openai.OpenAI(api_key=OPENAI_API_KEY)

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
anthropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
print(PINECONE_API_KEY)

pcsk_M7Zj7_J7Qd8oi8pVMRffngGjPUF7NumkpyQyLFfnZ5b6DxeebJ295BaKwqJGAdMGfT4pW


## **1. EXTRACT DATA**

In [26]:
import json
import requests
from bs4 import BeautifulSoup

In [27]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Extract all questions and answers from the “Questions-réponses sur le dispositif CEE” page
(https://www.ecologie.gouv.fr/politiques-publiques/questions-reponses-dispositif-cee)
and dump them into a structured JSON file.

Dependencies:
    - requests
    - beautifulsoup4

Usage:
    python extract_cee_qa.py
"""

'\nExtract all questions and answers from the “Questions-réponses sur le dispositif CEE” page\n(https://www.ecologie.gouv.fr/politiques-publiques/questions-reponses-dispositif-cee)\nand dump them into a structured JSON file.\n\nDependencies:\n    - requests\n    - beautifulsoup4\n\nUsage:\n    python extract_cee_qa.py\n'

In [28]:
def fetch_page(url):
    """
    Fetch the HTML content of the given URL.
    """
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()
    return resp.text

def extract_qa_pairs(html):
    """
    Parse the HTML with BeautifulSoup, find all <h4> tags whose text begins with "Q ",
    then collect everything between that <h4> and the next <h4> as the answer.
    Returns a list of dicts: [{'question': ..., 'answer': ...}, ...].
    """
    soup = BeautifulSoup(html, 'html.parser')
    qa_list = []

    # Find all <h4> tags
    all_h4 = soup.find_all('h4')
    for h4 in all_h4:
        text = h4.get_text(strip=True)
        # Only consider headings that start with "Q " (e.g. "Q I.1 – …")
        if not text.startswith('Q '):
            continue

        question_text = text

        # Collect all sibling nodes until the next <h4>
        answer_parts = []
        for sibling in h4.next_siblings:
            if getattr(sibling, 'name', None) == 'h4':
                # Reached the next question → stop collecting
                break

            # We only care about non-empty text nodes and tags
            if isinstance(sibling, str):
                s = sibling.strip()
                if s:
                    answer_parts.append(s)
            else:
                # If it's a tag, grab its text (stripping excess whitespace)
                s = sibling.get_text(separator=" ", strip=True).replace('\xa0', ' ')
                if s:
                    answer_parts.append(s)

        # Join all parts with two line breaks to preserve some structure
        answer_text = "\n".join(answer_parts).strip()
        #answer_text = " ".join(answer_parts).strip()

        qa_list.append({
            'question': question_text,
            'answer': answer_text
        })

    return qa_list

def main():
    url = "https://www.ecologie.gouv.fr/politiques-publiques/questions-reponses-dispositif-cee"
    html = fetch_page(url)
    qa_pairs = extract_qa_pairs(html)

    # Save to JSON file
    output_filename = "cee_questions_answers.json"
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(qa_pairs, f, ensure_ascii=False, indent=2)

    return html, qa_pairs

In [29]:
html, qa_pairs = main()

In [30]:
qa_pairs[10]['question']

"Q I.11 - Application des règles de la taxe sur la valeur ajoutée (TVA) aux cessions de certificats d’économies d'énergie"

In [31]:
qa_pairs[10]['answer'].split('\n')

['Cessions des certificats d’économies d’énergie',
 'Les certificats d’économies d’énergie (CEE) constituent des biens meubles incorporels négociables. A cet égard, l’article L. 221-8 du code de l’énergie dispose qu’ils peuvent être détenus, acquis ou cédés par toute personne mentionnée aux 1° à 6° de l’article L.  221-7 ou par toute autre personne morale.',
 "En matière de TVA, seules les livraisons de biens et de prestations de services réalisées à titre onéreux par un assujetti entrent dans le champ d'application de la taxe. Conformément à l’article 256 A du code général des impôts (CGI), sont considérées comme assujetties les personnes qui effectuent de manière indépendante une activité économique.",
 "La notion d'activité économique englobe toutes les activités de producteur, de commerçant ou de prestataire de services et notamment l’exploitation d'un bien meuble corporel ou incorporel, en vue d'en retirer des recettes ayant un caractère de permanence.",
 "Les cessions de CEE réal

## **2. VECTOR DB**

### **2.1. Initializing Pinecone and Creating an Index**

In [32]:
import pinecone_manager

In [33]:
# Load Pinecone manager
pc_manager = pinecone_manager.PineconeManager(
    api_key=PINECONE_API_KEY
)

In [34]:
# Create the indexe
profiles_index = pc_manager.create_index(
    "cee-questions-index",
    dimension=3072,
    delete=True
)

Index deleted cee-questions-index successfully.
Index created successfully.


### **2.2. Computing Embeddings & Upserting into Pinecone**

In [35]:
import embeddings_manager

In [36]:
# Load the embeddings generator
embeddings_gen = embeddings_manager.EmbeddingGenerator(
    openai_client=openai_client,
)

In [37]:
with open("cee_questions_answers.json", "r", encoding="utf-8") as f:
    qa_list = json.load(f)

In [38]:
def compute_db_data(data):
    idx = 0
    documents = []

    for item in tqdm(data):

        # Concat the question and answer into a single text
        text = f"Q: {item['question']}\nA: {item['answer']}"

        embeddings = embeddings_gen.create_embeddings(
            text,
            provider='openai',
            model='text-embedding-3-large'
        )

        documents.append({
            "id": f"qa_{idx}",
            "values": embeddings[0].astype(float).tolist(),
            "metadata": {"question": item["question"], "text": item["answer"]}
        })
        idx += 1

    return documents 

In [39]:
documents = compute_db_data(qa_list)

# Upsert the entries in the corresponding index
pc_manager.upsert_entries(
    entries=documents,
    index='cee-questions-index',
    namespace='cee-questions-answers',
)

100%|██████████| 109/109 [00:44<00:00,  2.46it/s]


## **3. RETRIEVAL**

In [40]:
# Laod Pinecone manager
pc_manager = pinecone_manager.PineconeManager(
    api_key=PINECONE_API_KEY
)

# Load index
cee_questions_index = pc_manager.get_index('cee-questions-index')

In [41]:
def retrieve_matches(query: str, index, namespace, top_k: int = 1) -> list[dict]:
    # Transform the query into embeddings
    query_vector = embeddings_gen.create_with_openai(query)
    query_vector = query_vector[0].tolist() 
        
    # Query the vector database
    result = index.query(
        namespace=namespace,
        vector=query_vector, 
        top_k=top_k,
        include_metadata=True,
        include_values=False
    )
    global db_match
    matches = []
    for db_match in result["matches"]:
        matches.append({
            "id": db_match["id"],
            "score": db_match["score"],
            "text": db_match.get("metadata", {})["text"]
        })
    return matches

In [47]:
QUERY = "Est-il possible de cumuler dans une même demande des opérations engagées en 3ème période et des opérations engagées en 4ème période"

top_matches = retrieve_matches(
    QUERY,
    index=cee_questions_index,
    namespace='cee-questions-answers',
)
top_matches

[{'id': 'qa_1',
  'score': 0.611982346,
  'text': 'L’article 8 de l’arrêté du 29 décembre 2014 relatif aux modalités d’application du dispositif des certificats d’économies d’énergie définit les volumes minimaux des demandes de certificats d’économies d’énergie :\n50 GWh cumac pour une demande portant sur des opérations standardisées ; 20 GWh cumac pour une demande portant sur des opérations spécifiques ; 20 GWh cumac pour une demande portant sur la contribution aux programmes.\nPour des raisons liées au traitement des demandes, les dossiers de demandes de CEE pour des opérations standardisées déposés à compter du 1 er janvier 2018 doivent contenir exclusivement des opérations engagées en troisième période ou des opérations engagées en quatrième période.\nCette contrainte liée au changement de période et de format des dossiers de demandes s’accompagne d’un assouplissement des modalités de dérogation prévues à l’article R.221-23 du code de l’énergie.\nLes dossiers de type spécifique et 

In [43]:
def generate_answer(matches, query):
    # Create context by combining the top matches
    context_pieces = []
    for i, doc in enumerate(matches, start=1):
        header = f"[Document {i} | ID={doc['id']} | Score={doc['score']:.4f}]"
        context_pieces.append(f"{header}\n{doc['text']}")
    context = "\n\n".join(context_pieces)

    # Generate the answer using OpenAI
    system_msg = {
        "role": "system",
        "content": (
            "Vous êtes un assistant expert en Certificats d'Économies d'Énergie (CEE). "
            "Répondez en français de manière précise, en vous appuyant sur le contexte fourni."
        )
    }
    user_msg = {
        "role": "user",
        "content": (
            "Voici des extraits (Q&A) tirés de la FAQ CEE :\n\n"
            f"{context}\n\n"
            "En vous basant uniquement sur ces extraits, répondez à la question suivante :\n"
            f"{query}\n\n"
            "Si l'information n'apparaît pas dans ces extraits, dites-le clairement."
            "Fournis une réponse concise et directe, sans ajouter d'informations supplémentaires."
        )
    }

    response = openai_client.responses.create(
        model="gpt-4.1",
        input=[system_msg, user_msg],
    )

    return response.output_text

In [44]:
answer = generate_answer(top_matches, QUERY)

In [45]:
print(answer)

L'information n'apparaît pas dans ces extraits.
